In [28]:
import numpy as np
from scipy import io

In [29]:
res101 = io.loadmat('../../resnet-feats/aPY/res101.mat')
att_splits = io.loadmat('att_splits.mat')
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'
prior_matrix = att_splits['att']
class_labels = res101['labels']
train_classes = class_labels[np.squeeze(att_splits[train_loc]-1)]
val_classes = class_labels[np.squeeze(att_splits[val_loc]-1)]
test_classes = class_labels[np.squeeze(att_splits[test_loc]-1)]
prior_matrix_tr = prior_matrix[:,(np.unique(train_classes)-1)]
prior_matrix_ts = prior_matrix[:,(np.unique(test_classes)-1)]

In [30]:
allclass_names=att_splits['allclasses_names']

In [31]:
train_img=res101['image_files'][np.squeeze(att_splits[train_loc]-1)]
val_img=res101['image_files'][np.squeeze(att_splits[val_loc]-1)]
test_img=res101['image_files'][np.squeeze(att_splits[test_loc]-1)]
train_img_names=[]
val_img_names=[]
test_img_names=[]
for i in range(train_img.shape[0]):
    train_img_names.append(train_img[i][0][0].split('ages/')[1]+'+'+allclass_names[train_classes[i][0]-1][0][0])
for i in range(test_img.shape[0]):
    test_img_names.append(test_img[i][0][0].split('ages/')[1]+'+'+allclass_names[test_classes[i][0]-1][0][0])
for i in range(val_img.shape[0]):
    val_img_names.append(val_img[i][0][0].split('ages/')[1]+'+'+allclass_names[val_classes[i][0]-1][0][0])

In [32]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b)
    print(len(a_set.intersection(b_set)))
    if len(a_set.intersection(b_set)) > 0: 
        return(True)  
    return(False)    

print(common_member(train_img_names, val_img_names)) 
print(common_member(train_img_names, test_img_names))
print(common_member(val_img_names, test_img_names))

0
False
0
False
0
False


In [33]:
all_img_names=train_img_names+val_img_names+test_img_names

In [34]:
rept_img_dict={}
for i in range(len(all_img_names)):
    rept_img_dict[all_img_names[i]]=0

In [35]:
img2att={}
apascal_train=open('./attribute_data/apascal_train.txt').readlines()
apascal_test=open('./attribute_data/apascal_test.txt').readlines()
ayahoo_test=open('./attribute_data/ayahoo_test.txt').readlines()
for i in range(len(apascal_train)):
    img_name=apascal_train[i].split(' ')[0]+'+'+apascal_train[i].split(' ')[1]
    if all_img_names.count(img_name)>1:
        suffix=str(rept_img_dict[img_name]+1)
        rept_img_dict[img_name]+=1
    else:
        suffix='1'
    bin_att=[]
    for j in range(64):
        bin_att.append(float(apascal_train[i].split(' ')[j+6]))
    img2att[img_name+'_'+suffix]=np.array(bin_att)
print(len(img2att))
for a in range(len(apascal_test)):
    img_name=apascal_test[a].split(' ')[0]+'+'+apascal_test[a].split(' ')[1]
    if all_img_names.count(img_name)>1:
        suffix=str(rept_img_dict[img_name]+1)
        rept_img_dict[img_name]+=1
    else:
        suffix='1'
    bin_att=[]
    for b in range(64):
        bin_att.append(float(apascal_test[a].split(' ')[b+6]))
    img2att[img_name+'_'+suffix]=np.array(bin_att)
print(len(img2att))
for x in range(len(ayahoo_test)):
    img_name=ayahoo_test[x].split(' ')[0]+'+'+ayahoo_test[x].split(' ')[1]
    if all_img_names.count(img_name)>1:
        suffix=str(rept_img_dict[img_name]+1)
        rept_img_dict[img_name]+=1
    else:
        suffix='1'
    bin_att=[]
    for y in range(64):
        bin_att.append(float(ayahoo_test[x].split(' ')[y+6]))
    img2att[img_name+'_'+suffix]=np.array(bin_att)
print(len(img2att))

6340
12695
15339


In [36]:
train_att_mat=np.zeros([len(train_img_names), 64])
count=0
for img_name in set(train_img_names):
    if rept_img_dict[img_name]!=0:
        for j in range(rept_img_dict[img_name]):
            train_att_mat[count+j]=img2att[img_name+'_'+str(j+1)]
        count+=rept_img_dict[img_name]
    else:
        train_att_mat[count]=img2att[img_name+'_1']
        count+=1

In [38]:
def diff_corr(corr_train, corr_test):
    dis_corr = (corr_train - corr_test)
    dis_corr = np.sign(corr_train)*dis_corr
    return dis_corr.clip(0,np.inf)

corr_train = np.corrcoef(train_att_mat.transpose())
nans = np.isnan(corr_train)
corr_train[nans] = 0

corr_test = np.corrcoef(prior_matrix_ts)
nans = np.isnan(corr_test)
corr_test[nans] = 0

dis_corr = diff_corr(corr_train, corr_test)
dis_corr_triu = np.triu(dis_corr,1)
loc = np.unravel_index(np.argsort(-dis_corr_triu, axis=None)[:100], dis_corr_triu.shape)
dis_corr_triu[loc]

array([1.06564791, 1.06271553, 1.05094655, 1.04861214, 1.04378   ,
       1.00750412, 0.9469992 , 0.90621382, 0.88490983, 0.85765022,
       0.85233419, 0.8519762 , 0.84394924, 0.83892886, 0.83506707,
       0.8078915 , 0.80687626, 0.80402085, 0.7935495 , 0.7934    ,
       0.79185654, 0.78029274, 0.76408187, 0.7534324 , 0.75225639,
       0.7428028 , 0.72167539, 0.7203806 , 0.71681475, 0.71177232,
       0.7110312 , 0.70909748, 0.70790235, 0.6846656 , 0.68408048,
       0.68315167, 0.66988935, 0.66869196, 0.66766969, 0.65730586,
       0.6406232 , 0.63498785, 0.63480464, 0.62591973, 0.62476026,
       0.62473087, 0.62035976, 0.6144634 , 0.61261097, 0.60273877,
       0.60168844, 0.59637401, 0.59349477, 0.57888844, 0.56403969,
       0.56150557, 0.55746645, 0.55166878, 0.53641877, 0.53142824,
       0.52552234, 0.52342315, 0.51891009, 0.51879968, 0.51698818,
       0.50643576, 0.4996758 , 0.49111273, 0.49062269, 0.48891981,
       0.4865874 , 0.48521468, 0.48250356, 0.47726794, 0.47694

In [39]:
dis_corr.mean()

0.07092237006514282

In [45]:
res101['image_files']

array([[array([u'/BS/Deep_Fragments/work/MSc/data/APY/images_att/VOCdevkit/VOC2008/JPEGImages/2008_000008.jpg'],
      dtype='<U92')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/APY/images_att/VOCdevkit/VOC2008/JPEGImages/2008_000008.jpg'],
      dtype='<U92')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/APY/images_att/VOCdevkit/VOC2008/JPEGImages/2008_000015.jpg'],
      dtype='<U92')],
       ...,
       [array([u'/BS/Deep_Fragments/work/MSc/data/APY/ayahoo_test_images/carriage_923.jpg'],
      dtype='<U72')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/APY/ayahoo_test_images/carriage_937.jpg'],
      dtype='<U72')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/APY/ayahoo_test_images/carriage_99.jpg'],
      dtype='<U71')]], dtype=object)

In [55]:
att_splits['train_loc']

array([[    3],
       [    4],
       [    5],
       ...,
       [14836],
       [14837],
       [14838]], dtype=uint16)

In [58]:
res101['labels']
# att_splits['allclasses_names']

array([[13],
       [15],
       [ 5],
       ...,
       [32],
       [32],
       [32]], dtype=uint8)

<br>
<br>
<br>
### New split with more correlation shift

In [15]:
res101 = io.loadmat('../../resnet-feats/aPY/res101.mat')
att_splits = io.loadmat('att_splits.mat')
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'
prior_matrix = att_splits['att']
class_labels = res101['labels']
train_classes = class_labels[np.squeeze(att_splits[train_loc]-1)]
val_classes = class_labels[np.squeeze(att_splits[val_loc]-1)]
test_classes = class_labels[np.squeeze(att_splits[test_loc]-1)]
prior_matrix_tr = prior_matrix[:,(np.unique(train_classes)-1)]
prior_matrix_ts = prior_matrix[:,(np.unique(test_classes)-1)]

In [23]:
len(att_splits['train_loc']) 

6086

In [34]:
def get_stats_of_data(fp_mat_file):
    att_splits = io.loadmat(fp_mat_file)
    print 'Train + Val count = ' + str(len(att_splits['train_loc']) + len(att_splits['val_loc']))
    print 'Test count = ' + str(len(att_splits['test_unseen_loc']))
    print 'Total = ' + str(len(att_splits['test_unseen_loc']) + len(att_splits['train_loc']) + len(att_splits['val_loc']))

In [43]:
get_stats_of_data('att_splits.mat')

Train + Val count = 7415
Test count = 7924
Total = 15339


In [75]:
len(img2att)

15339

In [76]:
def diff_corr(corr_train, corr_test):
    dis_corr = (corr_train - corr_test)
    dis_corr = np.sign(corr_train)*dis_corr
    return dis_corr.clip(0,np.inf)

In [143]:
def get_corr_shift(img2att, train_class, test_class):
    test_att_mat = []
    train_att_mat = []
    for img_name in img2att:
        cls = img_name.split('+')[1].split('_')[0]
        if cls in train_class:
            train_att_mat.append(img2att[img_name])
        elif cls in test_class:
            test_att_mat.append(img2att[img_name])
#     print '#Train instances = ' + str(len(train_att_mat)) + '\n#Test instances = ' + str(len(test_att_mat))
    
    train_att_mat = np.array(train_att_mat).transpose()
    test_att_mat = np.array(test_att_mat).transpose()
    
    corr_train = np.corrcoef(train_att_mat)
    corr_train[np.isnan(corr_train)] = 0.
    corr_test = np.corrcoef(test_att_mat)
    corr_test[np.isnan(corr_test)] = 0.
    
    dis_corr = diff_corr(corr_train, corr_test)
    
    # correlation shift score: 
    # 1) mean
    # corr_shift_score = np.mean(dis_corr)
    
    # 2) average of top n%
    dis_corr_array = dis_corr.flatten()
    top_percentage = 100
    num_elements = int((top_percentage/100.)*len(dis_corr_array))
    corr_shift_score = np.mean(dis_corr_array[np.argsort(dis_corr_array)[-num_elements:]])
    
    return corr_shift_score

In [85]:
all_classes = [cl[0][0] for cl in allclass_names]

In [105]:
import time

selected_classes = []
remaining_classes = all_classes[:]

start_time = time.time()
for i in range(12):
    best_cls = ''
    best_score = -1.
    for cls in remaining_classes:
        new_sel = selected_classes[:]
        new_sel.append(cls)
        new_rem = remaining_classes[:]
        new_rem.remove(cls)
        shift_score = get_corr_shift(img2att, new_rem, new_sel)
        if shift_score > best_score:
            best_score = shift_score
            best_cls = cls
    selected_classes.append(best_cls)
    remaining_classes.remove(best_cls)
    print str(i+1) + ') ' + selected_classes[-1] + ' -> ' + str(best_score) + ' :   ' + str(time.time() - start_time)+ 's' 

test_set = selected_classes[:]
train_val = remaining_classes[:]

1) bag -> 0.24254184334167883 :   1.83777809143s
2) bottle -> 0.24419044085401936 :   3.77944493294s
3) mug -> 0.2442747836824402 :   5.6346719265s
4) building -> 0.2437577141936315 :   7.31759190559s
5) tvmonitor -> 0.24232931874776364 :   9.14583110809s
6) diningtable -> 0.23958930677836193 :   11.0314850807s
7) car -> 0.23592500812921946 :   12.5436859131s
8) carriage -> 0.23595269829113152 :   14.1278250217s
9) bus -> 0.23646075010795312 :   15.5515639782s
10) train -> 0.23595360207898775 :   16.7958800793s
11) boat -> 0.2357936507909698 :   17.8987209797s
12) jetski -> 0.23392054184756544 :   18.9527161121s


In [106]:
train_val = all_classes[:]
for cls in all_classes:
    if cls in test_set:
        train_val.remove(cls) 
len(train_val)

20

In [108]:
selected_classes = []
remaining_classes = train_val[:]
for i in range(5):
    best_cls = ''
    best_score = -1.
    for cls in remaining_classes:
        new_sel = selected_classes[:]
        new_sel.append(cls)
        new_rem = remaining_classes[:]
        new_rem.remove(cls)
        shift_score = get_corr_shift(img2att, new_rem, test_set)
        if shift_score > best_score:
            best_score = shift_score
            best_cls = cls
    selected_classes.append(best_cls)
    remaining_classes.remove(best_cls)
    print str(i+1) + ') ' + selected_classes[-1] + ' -> ' + str(best_score)
train_set = remaining_classes[:]
val_set = selected_classes[:]

1) statue -> 0.23636153218187028
2) dog -> 0.23870271892078793
3) cat -> 0.24078543292743634
4) bird -> 0.24209048919771076
5) person -> 0.24616130378166207


In [144]:
get_corr_shift(img2att, train_set, test_set)

0.13281780813109434

In [131]:
tr = [apascal_train[i].split(' ')[1] for i in range(len(apascal_train))]

In [138]:
tr = np.unique([x.split('+')[1] for x in train_img_names])

In [141]:
te = np.unique([x.split('+')[1] for x in test_img_names])

In [145]:
get_corr_shift(img2att, tr, te)

0.07259888204453514

In [153]:
tr_count = 0
for img in all_img_names:
    n = img.split('+')[1]
    if n in train_set:
        tr_count += 1
print tr_count

te_count = 0
for img in all_img_names:
    n = img.split('+')[1]
    if n in test_set:
        te_count += 1
print te_count

val_count = 0
for img in all_img_names:
    n = img.split('+')[1]
    if n in val_set:
        val_count += 1
print val_count

4299
4349
6691


In [155]:
val_count + tr_count

10990

In [156]:
te_count

4349

In [157]:
split_dict = {}
split_dict['train_cls'] = train_set
split_dict['val_cls'] = val_set
split_dict['test_cls'] = test_set

import pickle
with open('apy_cs_split.npy', 'wb') as fp:
    np.save(fp, split_dict)

In [61]:
sd = np.load('apy_cs_split.npy', allow_pickle=True).item()
sd

{'test_cls': [u'bag',
  u'bottle',
  u'mug',
  u'building',
  u'tvmonitor',
  u'diningtable',
  u'car',
  u'carriage',
  u'bus',
  u'train',
  u'boat',
  u'jetski'],
 'train_cls': [u'aeroplane',
  u'bicycle',
  u'chair',
  u'cow',
  u'horse',
  u'motorbike',
  u'pottedplant',
  u'sheep',
  u'sofa',
  u'donkey',
  u'monkey',
  u'goat',
  u'wolf',
  u'zebra',
  u'centaur'],
 'val_cls': [u'statue', u'dog', u'cat', u'bird', u'person']}

In [65]:
all_classes = res101['labels']    
allclass_names=att_splits['allclasses_names']

train_loc = []
val_loc = []
test_loc = []

for i, label in enumerate(all_classes):
    if allclass_names[label-1] in sd['train_cls']:
        train_loc.append(i)
    elif allclass_names[label-1] in sd['val_cls']:
        val_loc.append(i)
    elif allclass_names[label-1] in sd['test_cls']:
        test_loc.append(i)

In [66]:
len(test_loc)

4349

In [64]:
allclass_names

array([[array([u'aeroplane'], dtype='<U9')],
       [array([u'bicycle'], dtype='<U7')],
       [array([u'bird'], dtype='<U4')],
       [array([u'boat'], dtype='<U4')],
       [array([u'bottle'], dtype='<U6')],
       [array([u'bus'], dtype='<U3')],
       [array([u'car'], dtype='<U3')],
       [array([u'cat'], dtype='<U3')],
       [array([u'chair'], dtype='<U5')],
       [array([u'cow'], dtype='<U3')],
       [array([u'diningtable'], dtype='<U11')],
       [array([u'dog'], dtype='<U3')],
       [array([u'horse'], dtype='<U5')],
       [array([u'motorbike'], dtype='<U9')],
       [array([u'person'], dtype='<U6')],
       [array([u'pottedplant'], dtype='<U11')],
       [array([u'sheep'], dtype='<U5')],
       [array([u'sofa'], dtype='<U4')],
       [array([u'train'], dtype='<U5')],
       [array([u'tvmonitor'], dtype='<U9')],
       [array([u'donkey'], dtype='<U6')],
       [array([u'monkey'], dtype='<U6')],
       [array([u'goat'], dtype='<U4')],
       [array([u'wolf'], dtype='<U4')],